In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dir_path = '/content/drive/Othercomputers/macbook-air/kaggle-ops/notebooks'
os.chdir(dir_path)

In [ ]:
! pip install japanize-matplotlib
! pip install shap

     |████████████████████████████████| 4.1 MB 3.7 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120275 sha256=6fde83f78484cf7d4d66d60dd8fb6dc7d0b18ea6557d6695f95cfdf0a5173a35
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib


In [ ]:
import glob
import sys,os
import json
import pprint
import time
import re
import datetime
import pickle
import string
import gc
import warnings
import yaml
import os
warnings.filterwarnings("ignore")
sys.path.append(os.pardir)
sys.path.append('../..')
sys.path.append('../../..')

import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import japanize_matplotlib # 日本語対応
import seaborn as sns
# pandasのオプション
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.3f}'.format
%matplotlib inline
# sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

from joblib import Parallel, delayed # よりお手軽にサクっと並列処理を実行出来るモジュール
from tqdm import tqdm, tqdm_notebook # プログレスバーを表示できる
from PIL import Image
tqdm.pandas()

# 外部モジュールを自動的にリロードする
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##  generate features

In [ ]:
! python ../code/10_titanic_fe.py

Survived
[Survived] start
[Survived] done in 0 s
PassengerId
[PassengerId] start
[PassengerId] done in 0 s
Pclass
[Pclass] start
[Pclass] done in 0 s
Name
[Name] start
[Name] done in 0 s
Sex
[Sex] start
[Sex] done in 0 s
Sex_label_encoder
[Sex_label_encoder] start
[Sex_label_encoder] done in 0 s
Age
[Age] start
[Age] done in 0 s
Age_mis_val_median
[Age_mis_val_median] start
[Age_mis_val_median] done in 0 s
SibSp
[SibSp] start
[SibSp] done in 0 s
Parch
[Parch] start
[Parch] done in 0 s
Family_size
[Family_size] start
[Family_size] done in 0 s
Ticket
[Ticket] start
[Ticket] done in 0 s
Fare
[Fare] start
[Fare] done in 0 s
Fare_mis_val_median
[Fare_mis_val_median] start
[Fare_mis_val_median] done in 0 s
Cabin
[Cabin] start
[Cabin] done in 0 s
Embarked
[Embarked] start
[Embarked] done in 0 s


## datasets確認

In [ ]:
CONFIG_FILE = '../configs/config.yaml'

with open(CONFIG_FILE) as file:
    yml = yaml.load(file)
MODEL_DIR_NAME = yml['SETTING']['MODEL_DIR_NAME']
FEATURE_DIR_NAME = yml['SETTING']['FEATURE_PATH']

FileNotFoundError: ignored

In [ ]:
def load_datasets_train(feats):
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    return X_train

def load_datasets_both(feats):
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_test.pkl') for f in feats]
    X_test = pd.concat(dfs, axis=1)
    return X_train, X_test

# 欠損値の確認
def missing_values_table(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
features = [
'age'
,'age_mis_val_median'
,'cabin'
,'embarked'
# ,'embarked_mis_val_s'
,'family_size'
,'fare'
,'fare_mis_val_median'
,'name'
,'parch'
,'passenger_id'
,'pclass'
,'sex'
,'sex_label_encoder'
,'sibsp'
,'survived'
,'ticket'
]

In [ ]:
train, test = load_datasets_both(features)

In [ ]:
display(train.shape, test.shape)

(891, 16)

(418, 15)

In [ ]:
display(train.head(), train.tail(), train.shape)

,Age,Age_mis_val_median,Cabin,Embarked,Family_Size,Fare,Fare_mis_val_median,Name,Parch,PassengerId,Pclass,Sex,sex_label_encoder,SibSp,Survived,Ticket
0,22.000,22.000,NaN,S,1,7.250,7.250,"Braund, Mr. Owen Harris",0,1,3,male,1,1,0,A/5 21171
1,38.000,38.000,C85,C,1,71.283,71.283,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",0,2,1,female,0,1,1,PC 17599
2,26.000,26.000,NaN,S,0,7.925,7.925,"Heikkinen, Miss. Laina",0,3,3,female,0,0,1,STON/O2. 3101282
3,35.000,35.000,C123,S,1,53.100,53.100,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,0,1,1,113803
4,35.000,35.000,NaN,S,0,8.050,8.050,"Allen, Mr. William Henry",0,5,3,male,1,0,0,373450


,Age,Age_mis_val_median,Cabin,Embarked,Family_Size,Fare,Fare_mis_val_median,Name,Parch,PassengerId,Pclass,Sex,sex_label_encoder,SibSp,Survived,Ticket
886,27.000,27.000,NaN,S,0,13.000,13.000,"Montvila, Rev. Juozas",0,887,2,male,1,0,0,211536
887,19.000,19.000,B42,S,0,30.000,30.000,"Graham, Miss. Margaret Edith",0,888,1,female,0,0,1,112053
888,NaN,28.000,NaN,S,3,23.450,23.450,"Johnston, Miss. Catherine Helen ""Carrie""",2,889,3,female,0,1,0,W./C. 6607
889,26.000,26.000,C148,C,0,30.000,30.000,"Behr, Mr. Karl Howell",0,890,1,male,1,0,1,111369
890,32.000,32.000,NaN,Q,0,7.750,7.750,"Dooley, Mr. Patrick",0,891,3,male,1,0,0,370376


(891, 16)

In [ ]:
display(train.describe(), test.describe())

,Age,Age_mis_val_median,Family_Size,Fare,Fare_mis_val_median,Parch,PassengerId,Pclass,sex_label_encoder,SibSp,Survived
count,714.000,891.000,891.000,891.000,891.000,891.000,891.000,891.000,891.000,891.000,891.000
mean,29.699,29.362,0.905,32.204,32.204,0.382,446.000,2.309,0.648,0.523,0.384
std,14.526,13.020,1.613,49.693,49.693,0.806,257.354,0.836,0.478,1.103,0.487
min,0.420,0.420,0.000,0.000,0.000,0.000,1.000,1.000,0.000,0.000,0.000
25%,20.125,22.000,0.000,7.910,7.910,0.000,223.500,2.000,0.000,0.000,0.000
50%,28.000,28.000,0.000,14.454,14.454,0.000,446.000,3.000,1.000,0.000,0.000
75%,38.000,35.000,1.000,31.000,31.000,0.000,668.500,3.000,1.000,1.000,1.000
max,80.000,80.000,10.000,512.329,512.329,6.000,891.000,3.000,1.000,8.000,1.000


,Age,Age_mis_val_median,Family_Size,Fare,Fare_mis_val_median,Parch,PassengerId,Pclass,sex_label_encoder,SibSp
count,332.000,418.000,418.000,417.000,418.000,418.000,418.000,418.000,418.000,418.000
mean,30.273,29.599,0.840,35.627,35.577,0.392,1100.500,2.266,0.636,0.447
std,14.181,12.704,1.519,55.908,55.850,0.981,120.810,0.842,0.482,0.897
min,0.170,0.170,0.000,0.000,0.000,0.000,892.000,1.000,0.000,0.000
25%,21.000,23.000,0.000,7.896,7.896,0.000,996.250,1.000,0.000,0.000
50%,27.000,27.000,0.000,14.454,14.454,0.000,1100.500,3.000,1.000,0.000
75%,39.000,35.750,1.000,31.500,31.472,0.000,1204.750,3.000,1.000,1.000
max,76.000,76.000,10.000,512.329,512.329,9.000,1309.000,3.000,1.000,8.000


In [ ]:
# 各データの欠損値を確認
display(
    missing_values_table(train),
    missing_values_table(test)
)

,Age,Age_mis_val_median,Cabin,Embarked,Family_Size,Fare,Fare_mis_val_median,Name,Parch,PassengerId,Pclass,Sex,sex_label_encoder,SibSp,Survived,Ticket
Total,177,0,687,2,0,0,0,0,0,0,0,0,0,0,0,0
Percent,19.865,0.000,77.104,0.224,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Types,float64,float64,object,object,int64,float64,float64,object,int64,int64,int64,object,int64,int64,int64,object


,Age,Age_mis_val_median,Cabin,Embarked,Family_Size,Fare,Fare_mis_val_median,Name,Parch,PassengerId,Pclass,Sex,sex_label_encoder,SibSp,Ticket
Total,86,0,327,0,0,1,0,0,0,0,0,0,0,0,0
Percent,20.574,0.000,78.230,0.000,0.000,0.239,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Types,float64,float64,object,object,int64,float64,float64,object,int64,int64,int64,object,int64,int64,object


## create submission

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
! python ../code/20_run.py

特徴量ディレクトリ:../data/features/ で実行しますか？[Y/n]
>> Y
[2022-02-06 03:15:34] - lgb_0206_0314 - train_x shape: (891, 7)
[2022-02-06 03:15:34] - lgb_0206_0314 - train_y shape: (891,)
[2022-02-06 03:15:34] - lgb_0206_0314 - start training cv
[2022-02-06 03:15:34] - lgb_0206_0314 - cv method: KFold
[2022-02-06 03:15:34] - lgb_0206_0314 fold 0 - start training
[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[LightGBM] [Info] Number of positive: 222, number of negative: 372
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data: 594, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.373737 -> initscore=-0.516216
[LightGBM] [Info] Start training from score -0.516216
[LightGBM] [Warning] No further splits wi